# nbdbt

> Use jupyter notebooks for dbt development

[`dbt`](https://getdbt.com) (data build tool) is revolutionizing the way we do the process of extracting value from our data. 

This package allows Jupyter notebooks to be used for developing dbt models and analyses in complement with other dbt command line tools and VS Code extensions.

## Install

`pip install git+https://github.com/butchland/nbdbt.git`

## How to use

The `%%dbt` cell magic allows you to create models and analyses in your dbt project.


In [ ]:
# hide
# notest
%reload_ext autoreload
%reload_ext lab_black
%autoreload 2

To use the `%%dbt` cellmagic in your notebook, you have to load the dbt cellmagic module first via `%load_ext` or `%reload_ext` line magics 

In [ ]:
# load dbt cell magic
%reload_ext nbdbt.dbt_cellmagic

In [ ]:
# hide
# delete my_third_model (if existing)
! rm -f ../my_dbt_project/models/my_third_model.sql
! rm -f ../my_dbt_project/target/compiled/my_dbt_project/models/my_third_model.sql

The `%dbtconfig` line magic configures a default project (and optionally the dbt profiles directory with `-d` flag) 

In [ ]:
%dbtconfig -p ../my_dbt_project

The next cell uses the `%%dbt` cell magic which will create a new model `my_third_model` and compile it as well.

In [ ]:
%%dbt -a my_third_model -n notebooks/index.ipynb models/my_third_model.sql
select *
from {{ ref('my_second_dbt_model') }}

We then assigned the result of the compilation to the `my_third_model` variable, which is a Dbt (cell) magic object

In [ ]:
# skip_test
my_third_model

In [ ]:
# hide
# skip_test
from pathlib import Path

assert Path("../my_dbt_project/models/my_third_model.sql").exists()
assert Path(
    "../my_dbt_project/target/compiled/my_dbt_project/models/my_third_model.sql"
).exists()

The `ref` method on `DbtMagicObject`  allows us to run the query and save the results into a dataframe.

In [ ]:
# skip_test
results = my_third_model.ref()
results  # dataframe

,id
0,1.0


The dbt magic object also has access to other useful properties (like the compiled sql used to create the results)

In [ ]:
# skip_test
print(my_third_model._compiled_sql)

-- AUTOGENERATED! DO NOT EDIT! File to edit: notebooks/index.ipynb (unless otherwise specified).
select *
from `sample-dbt-learn-project`.`jaffle_shop`.`my_second_dbt_model`


We can then run the usual _dbt_ commands to generate the model 

In [ ]:
# no_test
%cd ../my_dbt_project
! dbt run --select my_third_model
%cd ../nbs

/home/butch2/play/experiments/nbdbt/nbs
15:19:27  Running with dbt=1.1.1
15:19:27  Found 3 models, 4 tests, 0 snapshots, 3 analyses, 191 macros, 0 operations, 0 seed files, 0 sources, 0 exposures, 0 metrics
15:19:27  
15:19:28  Concurrency: 1 threads (target='dev')
15:19:28  
15:19:28  1 of 1 START view model jaffle_shop.my_third_model ............................. [RUN]
15:19:30  1 of 1 OK created view model jaffle_shop.my_third_model ........................ [OK in 1.32s]
15:19:30  
15:19:30  Finished running 1 view model in 3.23s.
15:19:30  
15:19:30  Completed successfully
15:19:30  
15:19:30  Done. PASS=1 WARN=0 ERROR=0 SKIP=0 TOTAL=1
/home/butch2/play/experiments/nbdbt/nbs


In [ ]:
import nbdbt.dbt_cellmagic as nbc

nbc.clear_cache()  # clears nbdtcache